In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install necessary packages
!pip install -q transformers
!pip install -q spacy
!pip install -q sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.0 MB/s eta 0:00:00


In [3]:
# Import required libraries
import os,shutil
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import random
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import torch

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(42)

Path of the folder containing transcript files

In [4]:
# Set the directory containing the transcription files
# directory = "/content/drive/MyDrive/transcription_files"
directory = "/content/drive/MyDrive/Transcription"

In [5]:
# Initialize a list to store file data
file_data = []

# Loop through each file in the directory
for file in os.listdir(directory):
    # Check if the file has a .txt extension
    if file.endswith("txt") :
        # Construct the full file path
        file_path = os.path.join(directory, file)
        # Open the file and read its content
        with open(file_path, "r") as f:
            text_doc = f.read()
            # Split the text document into smaller documents
            docs = re.split(r'\n\n',text_doc)
            # Extract the team name from the file name
            team = file.split(".")[0]
            
            # Combine the smaller documents so that each combined document is less than 512 tokens
            combined_docs = []
            current_doc = ""
            for doc in docs:
                if len(current_doc + doc) < 512:
                    current_doc += doc
                else:
                    combined_docs.append(current_doc)
                    current_doc = doc
            combined_docs.append(current_doc)
            
            # Add the combined documents and their corresponding team names to the file_data list
            for i, doc in enumerate(combined_docs):
                file_data.append({'Filenames': team, 'Texts': doc})

# Create a DataFrame from the file_data list
df = pd.DataFrame(file_data)
# Print DataFrame shape and unique file names
print(df.shape)
print(df.Filenames.unique())
# Display the first 10 rows of the DataFrame
df.head(10)

(232, 2)
['April2' 'April4' 'April9' 'April16' 'April25' 'March19' 'March28']


,Filenames,Texts
0,April2,"April 2nd, 2019. Say your numbers. 47, 48. Alr..."
1,April2,"What do you mean? It's still five, because it'..."
2,April2,Did you run a seven? Let me guess the number. ...
3,April2,and you still fucked up. Tadah. And the above ...
4,April2,What'd you put for one? Program one copies the...
5,April2,Five is false. Six is true. Okay. That's what ...
6,April2,ten names from the answer. You just search for...
7,April2,It's like we'd ask the question. I guess they ...
8,April2,"And then you can put in three names. Okay, jus..."
9,April2,"Oh, okay. But other than that, that's good. No..."


## T5

In [6]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-emotion")

In [7]:
# Create a deep copy of the original DataFrame
df1 = df.copy(deep=True)
# Initialize lists to store emotions and confidence scores
emotions = []
confidence = []

# Loop through each row in the DataFrame
for idx in tqdm(range(len(df1))):
    # Extract the speech text
    speech = df1.iloc[idx,1]
    try:
        # Tokenize the speech and generate an output label
        input_ids = tokenizer.encode_plus(speech, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
        output_label = model.generate(input_ids=input_ids['input_ids'], attention_mask=input_ids['attention_mask'], max_length=2)
        dec = [tokenizer.decode(ids) for ids in output_label]
        label = re.sub('<pad>','',dec[0])
        emotions.append(label.strip())
        # Calculate confidence scores for the emotion prediction
        output = model(input_ids=input_ids['input_ids'], attention_mask=input_ids['attention_mask'], decoder_input_ids=torch.ones((1, 1)).long())
        logits = output.logits.squeeze(0)
        probs = torch.softmax(logits, dim=-1)
        max_value, max_index = torch.max(probs, dim=1)
        score = round(max_value.item(),6)
        confidence.append(score)
    except Exception as e:
        # In case of any errors, append empty string and 0.0 for emotion and confidence, respectively
        emotions.append("")
        confidence.append(0.0)

# Add the predicted emotions and confidence scores to the DataFrame
df1["Emotions_T5"] = emotions
df1['Emotions_T5'].replace({'Joy': 'Happiness', 'Love': 'Disgust'}, inplace=True)
df1["Confidence_T5"] = confidence
df['Emotions_T5'] = df1['Emotions_T5']
df["Confidence_T5"] = df1["Confidence_T5"]

# Group the DataFrame by filenames and emotions
df1 = df1.groupby(['Filenames', 'Emotions_T5']).size().reset_index(name='Counts')
# Print the null values, DataFrame shape and unique emotions
print(df.isnull().sum(),df.shape,df1.isnull().sum(),df1.shape)


100%|██████████| 232/232 [20:27<00:00,  5.29s/it]

Filenames        0
Texts            0
Emotions_T5      0
Confidence_T5    0
dtype: int64 (232, 4) Filenames      0
Emotions_T5    0
Counts         0
dtype: int64 (30, 3)


In [8]:

# df['Emotions_T5'].replace({'joy': 'happiness', 'love': 'disgust'}, inplace=True)

In [9]:
df1.Emotions_T5.unique()

array(['anger', 'disgust', 'happiness', 'sadness', 'surprise', 'fear'],
      dtype=object)

In [10]:
# Define color scale for emotion categories
color_scale = {
    'Disgust': 'whitesmoke',
    'Happiness': 'springgreen',
    'Surprise': 'torquise',
    'Anger': 'crimson',
    'Sadness': 'black',
    'Fear': 'darkviolet'
}

# Create and display a pie chart of the T5 emotion distribution by filenames
fig = px.pie(df1, values='Counts', names='Emotions_T5', color='Emotions_T5', color_discrete_map=color_scale,
             labels={'Emotions_T5': 'Emotions_T5', 'Counts': 'Counts', 'Filenames': 'Filenames'},
             title='T5 Emotion Distribution by Filenames')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()


In [11]:
# Create and display a 3D scatter plot of the T5 emotion distribution by filenames
fig = px.scatter_3d(df1, x='Filenames', y='Emotions_T5', z='Counts',size='Counts',
                    color='Emotions_T5',title='T5 Emotion Distribution by Filenames')
fig.show()

In [12]:
# Create and display a 3D scatter plot of the T5 emotion confidence distribution by filenames
fig = px.scatter_3d(df, x='Filenames', y='Emotions_T5', z='Confidence_T5',size='Confidence_T5',
                    color='Emotions_T5',title='T5 Emotion Confidence Distribution by Filenames')
fig.show()

In [13]:
# Create and display a histogram of the T5 emotion distribution by filenames
fig = px.histogram(df, x="Emotions_T5", color="Filenames", title="T5 Emotion Distribution by Filenames",
                   labels={"Emotions_T5": "Emotions", "Counts": "Counts"}, width=800, 
                   category_orders={"Emotions_T5": df['Emotions_T5'].value_counts().index})

fig.update_layout(barmode="group", bargap=0.1, bargroupgap=0, xaxis={'showgrid': False, 'zeroline': False})

fig.show()

## BERT

In [14]:
# Import necessary libraries and models
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
from tqdm import tqdm
import tensorflow as tf

# Load tokenizer and model
tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")
# Create an emotion classification pipeline
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [15]:
# Initialize lists to store emotions and confidence values
emotions = []
confidence = []

# Create a copy of the original dataframe
df1 = df.copy(deep=True)
# Process each text in the dataframe and predict emotions and confidence
for idx in tqdm(range(len(df1))):
    speech = df1.iloc[idx,1]
    # Prepare input data, Perform predictions and store the results
    input_ids = []
    attention_masks = []
    for i in range(0, len(speech), 512):
        chunk = speech[i:i+512]
        encoded = tokenizer.encode_plus(chunk, add_special_tokens=True, max_length=512, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='tf')
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    if len(input_ids) > 1:
        input_ids = tf.concat(input_ids, axis=0)
        attention_masks = tf.concat(attention_masks, axis=0)
    elif len(input_ids) == 1:
        input_ids = input_ids[0]
        attention_masks = attention_masks[0]
    else:
        emotions.append("")
        confidence.append(0.0)
        continue

    try:
        outputs = model({'input_ids': input_ids, 'attention_mask': attention_masks})[0]
        predicted_class = tf.argmax(outputs, axis=1).numpy()[0]
        predicted_scores = tf.nn.softmax(outputs, axis=1).numpy()[0]
        emotions.append(model.config.id2label[predicted_class])
        confidence.append(predicted_scores[predicted_class])
    except:
        emotions.append("")
        confidence.append(0.0)
    
# Map the predicted emotions to a unified set
mappings = {
    'neutral' : 'fear',
    'admiration': 'happiness',
    'amusement': 'happiness',
    'anger': 'anger',
    'annoyance': 'anger',
    'approval': 'happiness',
    'caring': 'sadness',
    'confusion': 'surprise',
    'curiosity': 'surprise',
    'desire': 'happiness',
    'disappointment': 'sadness',
    'disapproval': 'anger',
    'disgust': 'disgust',
    'embarrassment': 'disgust',
    'excitement': 'happiness',
    'fear': 'fear',
    'gratitude': 'happiness',
    'grief': 'sadness',
    'joy': 'happiness',
    'love': 'happiness',
    'nervousness': 'fear',
    'optimism': 'happiness',
    'pride': 'happiness',
    'realization': 'surprise',
    'relief': 'happiness',
    'remorse': 'sadness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}

# Update the emotions list with the mapped values
emotions = [*map(mappings.get,emotions)]
# Add the predicted emotions and confidence values to the dataframe
df1["Emotions_BERT"] = np.array(emotions)
df1["Confidence_BERT"] = np.array(confidence)
df['Emotions_BERT'] = df1['Emotions_BERT']
df['Confidence_BERT'] = df1['Confidence_BERT']
# Group the dataframe by filenames, emotions, and confidence values
df1 = df1.groupby(['Filenames','Emotions_BERT','Confidence_BERT']).size().reset_index(name='Counts')
# Check for missing values and dataframe shapes
print(df.isnull().sum(),df.shape,df1.isnull().sum(),df1.shape)

100%|██████████| 232/232 [11:09<00:00,  2.89s/it]

Filenames          0
Texts              0
Emotions_T5        0
Confidence_T5      0
Emotions_BERT      0
Confidence_BERT    0
dtype: int64 (232, 6) Filenames          0
Emotions_BERT      0
Confidence_BERT    0
Counts             0
dtype: int64 (232, 4)


In [16]:
# Define a color scale for visualizations
color_scale = {
    'Fear': 'whitesmoke',
    'Happiness': 'springgreen',
    'Surprise': 'torquise',
    'Anger': 'crimson',
    'Sadness': 'black',
    'Disgust': 'darkviolet'
}

# Create and display a pie chart of the BERT emotion distribution by filenames
fig = px.pie(df1, values='Counts', names='Emotions_BERT', color='Emotions_BERT', 
             color_discrete_map=color_scale,color_discrete_sequence=px.colors.qualitative.Pastel,
             labels={'Emotions_BERT': 'Emotions_BERT', 'Counts': 'Counts', 'Filenames': 'Filenames'},
             title='BERT Emotion Distribution by Filenames')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()

In [17]:
# Create and display a 3D scatter plot of the BERT emotion distribution by filenames
fig = px.scatter_3d(df1, x='Filenames', y='Emotions_BERT', z='Counts',size='Counts',
                    color='Emotions_BERT',title='BERT Emotion Distribution by Filenames')
fig.show()

In [18]:
# Create and display a 3D scatter plot of the BERT emotion confidence distribution by filenames
fig = px.scatter_3d(df, x='Filenames', y='Emotions_BERT', z='Confidence_BERT',size='Confidence_BERT',
                    color='Emotions_BERT',title='BERT Emotion Confidence Distribution by Filenames')
fig.show()

In [19]:
# Create and display a histogram of the BERT emotion distribution by filenames
fig = px.histogram(df, x="Emotions_BERT", color="Filenames", title="BERT Emotion Distribution by Filenames",
                   labels={"Emotions_BERT": "Emotions", "Counts": "Counts"}, width=800, 
                   category_orders={"Emotions_BERT": df['Emotions_BERT'].value_counts().index})

fig.update_layout(barmode="group", bargap=0.1, bargroupgap=0, xaxis={'showgrid': False, 'zeroline': False})

fig.show()

## XLNet

In [20]:
# Import necessary libraries and models
from transformers import XLNetTokenizer, TFXLNetForSequenceClassification, pipeline

# Load tokenizer and model
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = TFXLNetForSequenceClassification.from_pretrained('xlnet-base-cased')

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning:

The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificat

In [21]:
# Create a copy of the original dataframe
df1 = df.copy(deep=True)
# Initialize lists to store emotions and confidence values
emotions_probs = []
emotion_labels = []

# Process each text in the dataframe and predict emotions and confidence
for idx in tqdm(range(len(df1))):
    speech = df1.iloc[idx,1]
    # Perform predictions and store the results
    try:
        input_ids = tokenizer.encode(speech,add_special_tokens=True,max_length=512,truncation=True,return_tensors='tf')
        output = model(input_ids=input_ids, training=False)
        dec = output.logits
        label = tf.argmax(dec, axis=1).numpy()[0]
        if label==0:
            emotions_probs.append(dec[0][0].numpy())
        else:
            emotions_probs.append(dec[0][1].numpy())
        label_pos_neg = {0:"negetive",1:"positive"}
        emotion_labels.append(label_pos_neg[label])
    except:
        emotion_labels.append("")
        emotions_probs.append(0.0)

# Add the predicted emotions and confidence values to the dataframe
df1["Emotions_XLNet"] = np.array(emotion_labels)
df1["Confidence_XLNet"] = np.array(emotions_probs)
df['Emotions_XLNet'] = df1['Emotions_XLNet']
df['Confidence_XLNet'] = df1['Confidence_XLNet']
# Group the dataframe by filenames, emotions, and confidence values
df1 = df1.groupby(['Filenames','Emotions_XLNet','Confidence_XLNet']).size().reset_index(name='Counts')

# Check for missing values and dataframe shapes
print(df.isnull().sum(),df.shape,df1.isnull().sum(),df1.shape)

100%|██████████| 232/232 [04:37<00:00,  1.19s/it]

Filenames           0
Texts               0
Emotions_T5         0
Confidence_T5       0
Emotions_BERT       0
Confidence_BERT     0
Emotions_XLNet      0
Confidence_XLNet    0
dtype: int64 (232, 8) Filenames           0
Emotions_XLNet      0
Confidence_XLNet    0
Counts              0
dtype: int64 (232, 4)


In [22]:
# Define a color scale for visualizations
color_scale = {
    'Positive': 'whitesmoke',
    'Negative': 'crimson'
}

# Create and display a pie chart of the XLNet emotion detection confidence based on each emotion category
fig = px.pie(df1, values='Counts', names='Emotions_XLNet', color='Emotions_XLNet', color_discrete_map=color_scale,
             labels={'Emotions_XLNet': 'Emotions_XLNet', 'Counts': 'Counts', 'Confidence_XLNet': 'Confindence_XLNet'},
             title='XLNet Emotion Detection Confidance based on Each Emotion Category')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()

In [23]:
# Create and display a 3D scatter plot of the XLNet emotion distribution by filenames
fig = px.scatter_3d(df1, x='Filenames', y='Emotions_XLNet', z='Counts',size='Counts',
                    color='Emotions_XLNet',title='XLNet Emotion Distribution by Filenames')
fig.show()

In [24]:
# Create and display a 3D scatter plot of the XLNet emotion confidence distribution by filenames
fig = px.scatter_3d(df, x='Filenames', y='Emotions_XLNet', z='Confidence_XLNet',
                    color='Emotions_XLNet',title='XLNet Emotion Confidence Distribution by Filenames')
fig.show()

In [25]:
# Create and display a histogram of the XLNet emotion distribution by filenames
fig = px.histogram(df, x="Emotions_XLNet", color="Filenames", title="XLNet Emotion Distribution by Filenames",
                   labels={"Emotions_XLNet": "Emotions", "Counts": "Counts"}, width=800, 
                   category_orders={"Emotions_XLNet": df['Emotions_XLNet'].value_counts().index})

fig.update_layout(barmode="group", bargap=0.1, bargroupgap=0, xaxis={'showgrid': False, 'zeroline': False})

fig.show()

In [26]:
# Save the final dataframe to an Excel file
df.to_excel('final_dataframe.xlsx', index=False)
# Read the saved Excel file and display the last 15 rows
df2 = pd.read_excel("final_dataframe.xlsx")
df2.tail(15)

,Filenames,Texts,Emotions_T5,Confidence_T5,Emotions_BERT,Confidence_BERT,Emotions_XLNet,Confidence_XLNet
217,March28,Sorry can I check your record?Yeah.Okay.Okay.O...,happiness,0.913248,sadness,0.688409,negetive,0.702565
218,March28,Is the highest number.Okay.So then index.But i...,surprise,0.954588,happiness,0.664699,negetive,0.686792
219,March28,Because it's the highest one.If that makes any...,anger,0.447777,anger,0.915274,negetive,0.388939
220,March28,So there's no nine.Gucci.So now we have to fin...,happiness,0.869049,sadness,0.363935,negetive,0.749700
221,March28,statement as that one.So right there.This righ...,happiness,0.862730,surprise,0.803543,negetive,0.792748
222,March28,I know you got kids.I'm older than you.How old...,happiness,0.879669,fear,0.620757,negetive,0.310657
223,March28,Is there anything you did not know about your ...,happiness,0.868714,anger,0.333641,negetive,0.777247
224,March28,But it's so monotonous.Oh should I? Okay.I'm f...,sadness,0.956441,surprise,0.606682,negetive,0.199783
225,March28,"No bracket on the if statement right there.No,...",happiness,0.939452,fear,0.795718,negetive,0.242704
226,March28,Your family don't care about you like that?The...,anger,0.982170,surprise,0.653232,negetive,0.498173


In [27]:
# Create and display a 3D scatter plot of all emotion distributions by filenames
fig = px.scatter_3d(df, x='Emotions_T5', y='Emotions_XLNet', z='Emotions_BERT',
                    color='Filenames',title='All Emotion Distribution by Filenames')
fig.show()

In [28]:
#